<a href="https://colab.research.google.com/github/nhamhung/Coder-School-Machine-Learning/blob/master/Week4_World_Cup_Google_Data_Studio_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


In [17]:
#importing libraries
import pandas as pd
import seaborn as sns
import itertools
import warnings
warnings.filterwarnings("ignore")
from wordcloud import WordCloud,STOPWORDS
import io
import base64
from matplotlib import rc,animation
from mpl_toolkits.mplot3d import Axes3D
# from mpl_toolkits.basemap import Basemap
import folium
import folium.plugins
import os


In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
winners = pd.read_csv('/content/drive/My Drive/FTMLE | 2020.05 | Philippine/Week 3/assignments/datasets/06-world-cup/winners.csv')
players = pd.read_csv('/content/drive/My Drive/FTMLE | 2020.05 | Philippine/Week 3/assignments/datasets/06-world-cup/players.csv')
matches = pd.read_csv('/content/drive/My Drive/FTMLE | 2020.05 | Philippine/Week 3/assignments/datasets/06-world-cup/matches.csv')

# Numberical data
For numberical data: How is the data distributed? How are they correlated? Provide summary statistics of these data (Review the slides Descriptive Statistics on Monday)? Identify outliers, check if they are errors or simply abnormalities in the data. ?

# Stat
 - What time most likely to score
 - Which team win the most Penalty
 - When Yellow and Red card are drawn on
 - Which team win most final, most matches
 - Which team score most goals
 - Which team have highest avg score per match
 - Ratio win of Home team vs away team
 - Where Referee come from
 - Who is the best goal scorer
 - Attendant of stadium
 - Number of goal in match
 - Top semi_final vs penalty
 - Goal Distribution by year


# Process and clean matches with 4 new columns:
- `Match Result`
- `Winner`
- `Loser`
- `Total Goals`

In [20]:
matches

,Year,Datetime,Stage,Stadium,City,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win conditions,Attendance,Half-time Home Goals,Half-time Away Goals,Referee,Assistant 1,Assistant 2,RoundID,MatchID,Home Team Initials,Away Team Initials
0,1930.0,13 Jul 1930 - 15:00,Group 1,Pocitos,Montevideo,France,4.0,1.0,Mexico,,4444.0,3.0,0.0,LOMBARDI Domingo (URU),CRISTOPHE Henry (BEL),REGO Gilberto (BRA),201.0,1096.0,FRA,MEX
1,1930.0,13 Jul 1930 - 15:00,Group 4,Parque Central,Montevideo,USA,3.0,0.0,Belgium,,18346.0,2.0,0.0,MACIAS Jose (ARG),MATEUCCI Francisco (URU),WARNKEN Alberto (CHI),201.0,1090.0,USA,BEL
2,1930.0,14 Jul 1930 - 12:45,Group 2,Parque Central,Montevideo,Yugoslavia,2.0,1.0,Brazil,,24059.0,2.0,0.0,TEJADA Anibal (URU),VALLARINO Ricardo (URU),BALWAY Thomas (FRA),201.0,1093.0,YUG,BRA
3,1930.0,14 Jul 1930 - 14:50,Group 3,Pocitos,Montevideo,Romania,3.0,1.0,Peru,,2549.0,1.0,0.0,WARNKEN Alberto (CHI),LANGENUS Jean (BEL),MATEUCCI Francisco (URU),201.0,1098.0,ROU,PER
4,1930.0,15 Jul 1930 - 16:00,Group 1,Parque Central,Montevideo,Argentina,1.0,0.0,France,,23409.0,0.0,0.0,REGO Gilberto (BRA),SAUCEDO Ulises (BOL),RADULESCU Constantin (ROU),201.0,1085.0,ARG,FRA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4568,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4569,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4570,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
import re

def match_result(winner, home_team):
  if winner == home_team:
    return "Home Win"
  elif winner != home_team:
    return "Away Win"
  else:
    return "Draw"

def winner(a, home, away):
  if "penalties" in a["Win conditions"]:
    if winner_condition(a["Win conditions"]) == "0":
      return a["Home Team Name"]
    else:
      return a["Away Team Name"]
  else:
    if home > away:
      return a["Home Team Name"]
    elif home < away:
      return a["Away Team Name"]
    else:
      return "Draw"

def loser(a, home, away):
  if "penalties" in a["Win conditions"]:
    if winner_condition(a["Win conditions"]) == "0":
      return a["Away Team Name"]
    else:
      return a["Home Team Name"]
  else:
    if home < away:
      return a["Home Team Name"]
    elif home > away:
      return a["Away Team Name"]
    else:
      return "Draw"

def winner_condition(x):
  lis = re.findall(r'\d{1,2}', str(x))
  if len(lis) > 1:
    if lis[0] > lis[1]:
      return "0"
    else:
      return "1"

# Remove duplicate rows with same MatchID column
matches = matches.drop_duplicates(subset="MatchID", keep="first")

# Remove rows where Year is null
matches = matches[matches["Year"].notnull()]

# Convert data to type `int`
matches["Home Team Goals"] = matches["Home Team Goals"].astype("int")
matches["Away Team Goals"] = matches["Away Team Goals"].astype("int")
matches["Half-time Home Goals"] = matches["Half-time Home Goals"].astype("int")
matches["Half-time Away Goals"] = matches["Half-time Away Goals"].astype("int")

# Further cleaning
matches.drop_duplicates(subset="MatchID", keep="first")
matches["Year"] = matches["Year"].astype(int)
matches["Attendance"] = matches["Attendance"].fillna("0")
matches["Attendance"] = matches["Attendance"].astype(int)

# Clean team names
matches["Home Team Name"] = matches["Home Team Name"].str.replace('rn">United Arab Emirates',"United Arab Emirates")
matches["Home Team Name"] = matches["Home Team Name"].str.replace("C�te d'Ivoire","Côte d’Ivoire")
matches["Home Team Name"] = matches["Home Team Name"].str.replace('rn">Republic of Ireland',"Republic of Ireland")
matches["Home Team Name"] = matches["Home Team Name"].str.replace('rn">Bosnia and Herzegovina',"Bosnia and Herzegovina")
matches["Home Team Name"] = matches["Home Team Name"].str.replace('rn">Serbia and Montenegro',"Serbia and Montenegro")
matches["Home Team Name"] = matches["Home Team Name"].str.replace('rn">Trinidad and Tobago',"Trinidad and Tobago")
matches["Home Team Name"] = matches["Home Team Name"].str.replace("Soviet Union","Russia")
matches["Home Team Name"] = matches["Home Team Name"].str.replace("Germany FR","Germany")

matches["Away Team Name"] = matches["Away Team Name"].str.replace('rn">United Arab Emirates',"United Arab Emirates")
matches["Away Team Name"] = matches["Away Team Name"].str.replace("C�te d'Ivoire","Côte d’Ivoire")
matches["Away Team Name"] = matches["Away Team Name"].str.replace('rn">Republic of Ireland',"Republic of Ireland")
matches["Away Team Name"] = matches["Away Team Name"].str.replace('rn">Bosnia and Herzegovina',"Bosnia and Herzegovina")
matches["Away Team Name"] = matches["Away Team Name"].str.replace('rn">Serbia and Montenegro',"Serbia and Montenegro")
matches["Away Team Name"] = matches["Away Team Name"].str.replace('rn">Trinidad and Tobago',"Trinidad and Tobago")
matches["Away Team Name"] = matches["Away Team Name"].str.replace("Germany FR","Germany")
matches["Away Team Name"] = matches["Away Team Name"].str.replace("Soviet Union","Russia")

# Create 4 new columns
matches["Winner"] = matches.apply(lambda match: winner(match, match["Home Team Goals"], match["Away Team Goals"]), axis=1)
matches["Loser"] = matches.apply(lambda match:loser(match, match["Home Team Goals"], match["Away Team Goals"]), axis=1)
matches["Match Result"] = matches.apply(lambda match: match_result(match["Winner"], match["Home Team Name"]), axis=1)
matches["Total Goals"] = matches["Home Team Goals"] + matches["Away Team Goals"]

In [22]:
matches[matches["Win conditions"] != " "]

,Year,Datetime,Stage,Stadium,City,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win conditions,Attendance,Half-time Home Goals,Half-time Away Goals,Referee,Assistant 1,Assistant 2,RoundID,MatchID,Home Team Initials,Away Team Initials,Winner,Loser,Match Result,Total Goals
18,1934,27 May 1934 - 16:30,Preliminary round,Stadio Benito Mussolini,Turin,Austria,3,2,France,Austria win after extra time,16000,0,0,VAN MOORSEL Johannes (NED),CAIRONI Camillo (ITA),BAERT Louis (BEL),204.0,1104.0,AUT,FRA,Austria,France,Home Win,5
34,1934,10 Jun 1934 - 17:30,Final,Nazionale PNF,Rome,Italy,2,1,Czechoslovakia,Italy win after extra time,55000,0,0,EKLIND Ivan (SWE),BAERT Louis (BEL),IVANCSICS Mihaly (HUN),3490.0,1134.0,ITA,TCH,Italy,Czechoslovakia,Home Win,3
39,1938,05 Jun 1938 - 17:00,First round,Stade V�lodrome,Marseilles,Italy,2,1,Norway,Italy win after extra time,19000,0,0,BERANEK Alois (AUT),BOUTOURE D. (FRA),TREHOU D. (FRA),206.0,1179.0,ITA,NOR,Italy,Norway,Home Win,3
40,1938,05 Jun 1938 - 17:30,First round,Stade de la Meinau,Strasbourg,Brazil,6,5,Poland,Brazil win after extra time,13452,0,0,EKLIND Ivan (SWE),POISSANT Louis (FRA),KISSENBERGER Ernest (FRA),206.0,1150.0,BRA,POL,Brazil,Poland,Home Win,11
41,1938,05 Jun 1938 - 18:30,First round,Cavee Verte,Le Havre,Czechoslovakia,3,0,Netherlands,Czechoslovakia win after extra time,11000,0,0,LECLERCQ Lucien (FRA),OLIVE D. (FRA),SDEZ Victor (FRA),206.0,1172.0,TCH,NED,Czechoslovakia,Netherlands,Home Win,3
98,1954,30 Jun 1954 - 18:00,Semi-finals,La Pontaise,Lausanne,Hungary,4,2,Uruguay,Hungary win after extra time,45000,0,0,GRIFFITHS Benjamin (WAL),FAULTLESS Charlie (SCO),VINCENTI Raymond (FRA),462.0,1295.0,HUN,URU,Hungary,Uruguay,Home Win,6
126,1958,17 Jun 1958 - 19:00,Group 1,Malmo Stadion,Malm�,Northern Ireland,2,1,Czechoslovakia,Northern Ireland win after extra time,6196,0,0,GUIGUE Maurice (FRA),FERNANDES CAMPOS Joaquim (POR),AHLNER Sten (SWE),220.0,1422.0,NIR,TCH,Northern Ireland,Czechoslovakia,Home Win,3
199,1966,30 Jul 1966 - 15:00,Final,Wembley Stadium,London,England,4,2,Germany,England win after extra time,96924,0,0,DIENST Gottfried (SUI),BAKHRAMOV Tofik (URS),GALBA Karol (TCH),3478.0,1633.0,ENG,FRG,England,Germany,Home Win,6
225,1970,14 Jun 1970 - 12:00,Quarter-finals,Nou Camp - Estadio Le�n,Leon,Germany,3,2,England,Germany FR win after extra time,23357,0,0,COEREZZA Norberto Angel (ARG),VELASQUEZ Guillermo (COL),ORTIZ DE MENDIBIL Jose Maria (ESP),251.0,1811.0,FRG,ENG,Germany,England,Home Win,5
227,1970,14 Jun 1970 - 12:00,Quarter-finals,Estadio Azteca,Mexico City,Uruguay,1,0,Russia,Uruguay win after extra time,26085,0,0,VAN RAVENS Laurens (NED),DAVIDSON Bob (SCO),GLOECKNER Rudolf (GDR),251.0,1925.0,URU,URS,Uruguay,Russia,Home Win,1


In [23]:
players.sample(10)

,RoundID,MatchID,Team Initials,Coach Name,Line-up,Shirt Number,Player Name,Position,Event
9604,250,1752,MEX,CARDENAS Raul (MEX),N,16,Isidoro DIAZ,NaN,NaN
650,405,1087,ARG,OLAZAR Francisco (ARG),S,0,Juan EVARISTO,NaN,NaN
13606,293,741,ALG,MEKHLOUFI Rachid (ALG),N,19,Djamel TLEMCANI,NaN,NaN
5385,220,1408,WAL,MURPHY James (WAL),N,17,Ken LEEK,NaN,NaN
24661,1024,8776,PAR,CARPEGIANI (BRA),S,2,ARCE,NaN,Y84'
29458,97410100,97410028,CRO,KRANJCAR Zlatko (CRO),N,23,BUTINA,NaN,NaN
7185,238,1636,ENG,RAMSEY Alf (ENG),N,14,Jimmy ARMFIELD,NaN,NaN
16191,308,571,FRA,MICHEL Henri (FRA),N,13,Bernard GENGHINI,NaN,NaN
12284,278,2408,SCO,MacLEOD Alistair (SCO),N,13,Stuart KENNEDY,NaN,NaN
1212,418,1123,ESP,GARCIA Amadeo (ESP),N,0,MARCULETA,NaN,NaN


# Top 10 Teams in World Cup Performance

In [24]:
home = matches["Home Team Name"].value_counts().reset_index()
home.columns = ["Team", "Count"]
away = matches["Away Team Name"].value_counts().reset_index()
away.columns = ["Team", "Count"]

combined_matches = pd.concat([home, away], axis=0)

combined_matches = combined_matches.groupby("Team")["Count"].sum().reset_index().sort_values("Count", ascending=False)

# top 10 teams with the most number of world cup matches
top_10_matches = combined_matches.head(10)
top_10_matches

,Team,Count
29,Germany,106
8,Brazil,104
39,Italy,83
2,Argentina,77
26,England,62
67,Spain,59
27,France,59
45,Mexico,53
77,Uruguay,51
47,Netherlands,50


In [25]:
home_team_goals = matches.groupby("Home Team Name")["Home Team Goals"].sum().reset_index()
home_team_goals.columns = ["Team", "Goal"]
away_team_goals = matches.groupby("Away Team Name")["Away Team Goals"].sum().reset_index()
away_team_goals.columns = ["Team", "Goal"]

combined_goals = pd.concat([home_team_goals, away_team_goals], axis=0)
combined_goals = combined_goals.groupby("Team")["Goal"].sum().reset_index().sort_values("Goal", ascending=False)

# top 10 teams with the most number of world cup goals
top_10_goals = combined_goals.head(10)

top_10_goals

,Team,Goal
29,Germany,224
8,Brazil,221
2,Argentina,131
39,Italy,128
27,France,106
67,Spain,92
34,Hungary,87
47,Netherlands,86
77,Uruguay,80
26,England,79


In [26]:
winners["Winner"] = winners["Winner"].str.replace("Germany FR", "Germany")
winners["Runners-Up"] = winners["Runners-Up"].str.replace("Germany FR", "Germany")
winners["Third"] = winners["Third"].str.replace("Germany FR", "Germany")
winners["Fourth"] = winners["Fourth"].str.replace("Germany FR", "Germany")


In [27]:
players

,RoundID,MatchID,Team Initials,Coach Name,Line-up,Shirt Number,Player Name,Position,Event
0,201,1096,FRA,CAUDRON Raoul (FRA),S,0,Alex THEPOT,GK,NaN
1,201,1096,MEX,LUQUE Juan (MEX),S,0,Oscar BONFIGLIO,GK,NaN
2,201,1096,FRA,CAUDRON Raoul (FRA),S,0,Marcel LANGILLER,NaN,G40'
3,201,1096,MEX,LUQUE Juan (MEX),S,0,Juan CARRENO,NaN,G70'
4,201,1096,FRA,CAUDRON Raoul (FRA),S,0,Ernest LIBERATI,NaN,NaN
...,...,...,...,...,...,...,...,...,...
37779,255959,300186501,ARG,SABELLA Alejandro (ARG),N,19,ALVAREZ,NaN,NaN
37780,255959,300186501,GER,LOEW Joachim (GER),N,6,KHEDIRA,NaN,NaN
37781,255959,300186501,ARG,SABELLA Alejandro (ARG),N,20,AGUERO,NaN,IH46' Y65'
37782,255959,300186501,GER,LOEW Joachim (GER),N,21,MUSTAFI,NaN,NaN


In [28]:
events = players[players["Event"].notnull()]
events = events[["Team Initials", "Player Name", "Position", "Event"]]

def extract_minute(event):
  event_lis = [x for x in event.split(" ")]
  minutes = []
  for x in event_lis:
    digits = [i for i in x if i.isdigit()]
    minutes.append("".join(digits))
  return minutes

def is_goal(event):
  if event[0] == "G" or event[0:2] == "OG" or event[0] == "P":
    return "Goal"
  else:
    return "Not Goal"

def insert_minutes(lis, minutes):
  if len(lis) > 1:
    for x in lis:
      minutes.append(x)
  else:
    minutes.append(lis[0])

events["Score"] = events.apply(lambda x:is_goal(x["Event"]), axis=1)
events["Minutes"] = events.apply(lambda x:extract_minute(x["Event"]), axis=1)
events = events[events["Score"] == "Goal"]
events = events[["Minutes"]]

minutes = []
events.apply(lambda event:insert_minutes(event["Minutes"], minutes), axis=1)

minutes = pd.DataFrame(minutes, columns=["Minutes"])
minutes["Minutes"] = minutes["Minutes"].astype(int)
minutes = minutes["Minutes"].value_counts().reset_index()
minutes.columns = ["Minutes", "Count"]
minutes = minutes.sort_values("Minutes", ascending=True)

In [29]:
minutes.to_csv('minutes.csv', sep=',')

In [30]:
matches.to_csv('matches.csv', sep=",")

In [31]:
players.to_csv('players.csv', sep=",")

In [32]:
winners.to_csv('winner.csv', sep=',')

In [33]:
def wc_score(stage):
    stage = stage.lower()
    if 'group' in stage:
        return 1
    elif 'round' in stage:
        return 2
    elif 'quarter' in stage:
        return 3
    elif 'semi' in stage:
        return 4
    elif 'third' in stage:
        return 5
    else:
        return 8

In [34]:
matches['score'] = matches['Stage'].apply(wc_score)

In [35]:
worldcup_score = matches.groupby('Winner')['score'].sum().reset_index()
worldcup_score.sort_values(by='score', ascending=False, inplace=True)
worldcup_score = worldcup_score[worldcup_score['Winner'] !='Draw']

worldcup_score.set_index('Winner', inplace=True)

In [36]:
worldcup_score

,score
Winner,
Germany,166
Brazil,158
Italy,114
Argentina,84
France,64
...,...
Tunisia,1
Czech Republic,1
Bosnia and Herzegovina,1


In [37]:
worldcup_score.to_csv('Worldcup_score.csv')

# Team Performance

In [38]:
combined_matches

,Team,Count
29,Germany,106
8,Brazil,104
39,Italy,83
2,Argentina,77
26,England,62
...,...,...
13,China PR,3
11,Canada,3
7,Bosnia and Herzegovina,3
80,Zaire,3


In [39]:
lost = matches["Loser"].value_counts().reset_index()

In [40]:
lost

,index,Loser
0,Draw,160
1,Mexico,27
2,Argentina,22
3,France,21
4,Italy,20
...,...,...
77,Cuba,1
78,Wales,1
79,Senegal,1
80,Israel,1


In [41]:
win = matches["Winner"].value_counts().reset_index()

In [42]:
win

,index,Winner
0,Draw,160
1,Brazil,73
2,Germany,70
3,Argentina,46
4,Italy,46
...,...,...
58,Iran,1
59,Korea DPR,1
60,Bosnia and Herzegovina,1
61,Tunisia,1


In [43]:
win_lost = win.merge(lost, left_on="index", right_on="index", how="left")

In [44]:
win_lost = win_lost[win_lost["index"] != "Draw"]
win_lost.columns = ["Team", "Win", "Lose"]

In [45]:
win_lost

,Team,Win,Lose
1,Brazil,73,18
2,Germany,70,20
3,Argentina,46,22
4,Italy,46,20
5,Spain,30,20
...,...,...,...
58,Iran,1,4
59,Korea DPR,1,5
60,Bosnia and Herzegovina,1,2
61,Tunisia,1,7


In [46]:
wl = win_lost.copy()

wl = wl.merge(combined_matches, left_on="Team", right_on="Team", how="left")

In [47]:
wl["Draw"] = wl["Count"] - (wl["Win"] + wl["Lose"])

In [48]:
wl.index = wl.Team

In [49]:
wl = wl.sort_values(by="Win", ascending=False)

In [50]:
wl

,Team,Win,Lose,Count,Draw
Team,,,,,
Brazil,Brazil,73,18,104,13
Germany,Germany,70,20,106,16
Argentina,Argentina,46,22,77,9
Italy,Italy,46,20,83,17
Spain,Spain,30,20,59,9
...,...,...,...,...,...
Iran,Iran,1,4,6,1
Korea DPR,Korea DPR,1,5,7,1
Bosnia and Herzegovina,Bosnia and Herzegovina,1,2,3,0


In [51]:
lost

,index,Loser
0,Draw,160
1,Mexico,27
2,Argentina,22
3,France,21
4,Italy,20
...,...,...
77,Cuba,1
78,Wales,1
79,Senegal,1
80,Israel,1


In [52]:
matches_played = combined_matches.copy()

matches_info = matches_played.merge(lost, left_on="Team", right_on="index", how="left")
matches_info = matches_info.merge(win, left_on="Team", right_on="index", how="left")


In [53]:
matches_info = matches_info[['Team', 'Count', 'Loser', 'Winner']]
matches_info.columns = ['Team', 'Matches', 'Lose', 'Win']

In [54]:
matches_info.isnull().sum()
matches_info = matches_info.fillna(0)

In [55]:
matches_info['Win'] = matches_info['Win'].astype(int)
matches_info['Draw'] = matches_info['Matches'] - (matches_info['Win'] + matches_info['Lose'])


In [56]:
matches_info = matches_info.merge(worldcup_score, left_on='Team', right_on='Winner', how='left')

In [57]:
matches_info['score'] = matches_info['score'].fillna(0)
matches_info['score'] = matches_info['score'].astype(int)

In [58]:
matches_info = matches_info.merge(combined_goals, left_on='Team', right_on='Team', how='left')

In [59]:
matches_info.columns = ['team', 'matches', 'lose', 'win', 'draw', 'score', 'goals']

In [60]:
matches_info['avg_goals_score'] = matches_info['goals'] / matches_info['matches']

In [61]:
matches_info['avg_goals_score'] = matches_info['avg_goals_score'].round(2)

In [62]:
matches_info['avg_goals_score'] = matches_info['avg_goals_score'].fillna(0)

In [63]:
matches_info

,team,matches,lose,win,draw,score,goals,avg_goals_score
0,Germany,106,20,70,16,166,224,2.11
1,Brazil,104,18,73,13,158,221,2.12
2,Italy,83,20,46,17,114,128,1.54
3,Argentina,77,22,46,9,84,131,1.70
4,England,62,19,26,17,44,79,1.27
...,...,...,...,...,...,...,...,...
76,China PR,3,3,0,0,0,0,0.00
77,Canada,3,3,0,0,0,0,0.00
78,Bosnia and Herzegovina,3,2,1,0,1,4,1.33
79,Zaire,3,3,0,0,0,0,0.00


# Average goals scored per match
# Average goals being scored per match

In [64]:
test = matches.copy()

In [65]:
test

,Year,Datetime,Stage,Stadium,City,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win conditions,Attendance,Half-time Home Goals,Half-time Away Goals,Referee,Assistant 1,Assistant 2,RoundID,MatchID,Home Team Initials,Away Team Initials,Winner,Loser,Match Result,Total Goals,score
0,1930,13 Jul 1930 - 15:00,Group 1,Pocitos,Montevideo,France,4,1,Mexico,,4444,3,0,LOMBARDI Domingo (URU),CRISTOPHE Henry (BEL),REGO Gilberto (BRA),201.0,1096.0,FRA,MEX,France,Mexico,Home Win,5,1
1,1930,13 Jul 1930 - 15:00,Group 4,Parque Central,Montevideo,USA,3,0,Belgium,,18346,2,0,MACIAS Jose (ARG),MATEUCCI Francisco (URU),WARNKEN Alberto (CHI),201.0,1090.0,USA,BEL,USA,Belgium,Home Win,3,1
2,1930,14 Jul 1930 - 12:45,Group 2,Parque Central,Montevideo,Yugoslavia,2,1,Brazil,,24059,2,0,TEJADA Anibal (URU),VALLARINO Ricardo (URU),BALWAY Thomas (FRA),201.0,1093.0,YUG,BRA,Yugoslavia,Brazil,Home Win,3,1
3,1930,14 Jul 1930 - 14:50,Group 3,Pocitos,Montevideo,Romania,3,1,Peru,,2549,1,0,WARNKEN Alberto (CHI),LANGENUS Jean (BEL),MATEUCCI Francisco (URU),201.0,1098.0,ROU,PER,Romania,Peru,Home Win,4,1
4,1930,15 Jul 1930 - 16:00,Group 1,Parque Central,Montevideo,Argentina,1,0,France,,23409,0,0,REGO Gilberto (BRA),SAUCEDO Ulises (BOL),RADULESCU Constantin (ROU),201.0,1085.0,ARG,FRA,Argentina,France,Home Win,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
831,2014,05 Jul 2014 - 13:00,Quarter-finals,Estadio Nacional,Brasilia,Argentina,1,0,Belgium,,68551,1,0,Nicola RIZZOLI (ITA),Renato FAVERANI (ITA),Andrea STEFANI (ITA),255953.0,300186504.0,ARG,BEL,Argentina,Belgium,Home Win,1,3
832,2014,29 Jun 2014 - 13:00,Round of 16,Estadio Castelao,Fortaleza,Netherlands,2,1,Mexico,,58817,0,0,PROENCA Pedro (POR),MIRANDA Bertino (POR),TRIGO Jose (POR),255951.0,300186508.0,NED,MEX,Netherlands,Mexico,Home Win,3,2
833,2014,29 Jun 2014 - 17:00,Round of 16,Arena Pernambuco,Recife,Costa Rica,1,1,Greece,Costa Rica win on penalties (5 - 3),41242,0,0,Ben WILLIAMS (AUS),CREAM Matthew (AUS),ANAZ Hakan (AUS),255951.0,300186459.0,CRC,GRE,Costa Rica,Greece,Home Win,2,2
834,2014,01 Jul 2014 - 13:00,Round of 16,Arena de Sao Paulo,Sao Paulo,Argentina,1,0,Switzerland,Argentina win after extra time,63255,0,0,ERIKSSON Jonas (SWE),KLASENIUS Mathias (SWE),WARNMARK Daniel (SWE),255951.0,300186503.0,ARG,SUI,Argentina,Switzerland,Home Win,1,2


In [66]:
def goals_scored(match, loser, home, away_goals, home_goals):
  if loser == home:
    return away_goals
  else:
    return home_goals

In [67]:
test['Lost Goals'] = test.apply(lambda x: goals_scored(x, x['Loser'], x['Home Team Name'], x['Away Team Goals'], x['Home Team Goals']), axis=1)

In [68]:
test = test[['Loser', 'Lost Goals']]

In [69]:
test = test.groupby('Loser')['Lost Goals'].sum()

In [70]:
test = test.to_frame()

In [71]:
matches_info = matches_info.merge(test, left_on='team', right_on='Loser', how='left')

In [72]:
matches_info = matches_info.rename(columns={'Lost Goals': 'lost_goals'})

In [73]:
matches_info['avg_goals_lost'] = (matches_info['lost_goals'] / matches_info['matches']).round(2)

# Participation

In [74]:
part1 = matches[['Year', 'Home Team Name']]
part1.columns = ['year', 'team']

part2 = matches[['Year', 'Away Team Name']]
part2.columns = ['year', 'team']


In [75]:
part = pd.concat([part1, part2], axis=0)

In [76]:
part

,year,team
0,1930,France
1,1930,USA
2,1930,Yugoslavia
3,1930,Romania
4,1930,Argentina
...,...,...
831,2014,Belgium
832,2014,Mexico
833,2014,Greece
834,2014,Switzerland


In [77]:
part = part.groupby(['year', 'team']).count().reset_index()
part = part['team'].value_counts().reset_index()
part.columns = ['team', 'participation']

In [78]:
matches_info = matches_info.merge(part, left_on='team', right_on='team', how='left')

In [79]:
matches_info

,team,matches,lose,win,draw,score,goals,avg_goals_score,lost_goals,avg_goals_lost,participation
0,Germany,106,20,70,16,166,224,2.11,58,0.55,18
1,Brazil,104,18,73,13,158,221,2.12,45,0.43,20
2,Italy,83,20,46,17,114,128,1.54,36,0.43,18
3,Argentina,77,22,46,9,84,131,1.70,52,0.68,16
4,England,62,19,26,17,44,79,1.27,36,0.58,14
...,...,...,...,...,...,...,...,...,...,...,...
76,China PR,3,3,0,0,0,0,0.00,9,3.00,1
77,Canada,3,3,0,0,0,0,0.00,5,1.67,1
78,Bosnia and Herzegovina,3,2,1,0,1,4,1.33,3,1.00,1
79,Zaire,3,3,0,0,0,0,0.00,14,4.67,1


# Championship

In [80]:
final = winners['Winner'].value_counts().reset_index()

In [81]:
final.columns = ['team', 'champion']

In [82]:
runner_up = winners['Runners-Up'].value_counts().reset_index()

In [83]:
runner_up.columns = ['team', 'runners_up']

In [84]:
third = winners['Third'].value_counts().reset_index()
third.columns = ['team', 'third']

In [85]:
fourth = winners['Fourth'].value_counts().reset_index()
fourth.columns = ['team', 'fourth']

In [86]:
matches_info = matches_info.merge(final, left_on='team', right_on='team', how='left')
matches_info = matches_info.merge(runner_up, left_on='team', right_on='team', how='left')
matches_info = matches_info.merge(third, left_on='team', right_on='team', how='left')
matches_info = matches_info.merge(fourth, left_on='team', right_on='team', how='left')

In [87]:
matches_info = matches_info.fillna(0)

In [88]:
matches_info['champion'] = matches_info['champion'].astype(int)
matches_info['runners_up'] = matches_info['runners_up'].astype(int)
matches_info['third'] = matches_info['third'].astype(int)
matches_info['fourth'] = matches_info['fourth'].astype(int)

In [89]:
matches_info

,team,matches,lose,win,draw,score,goals,avg_goals_score,lost_goals,avg_goals_lost,participation,champion,runners_up,third,fourth
0,Germany,106,20,70,16,166,224,2.11,58,0.55,18,4,4,4,1
1,Brazil,104,18,73,13,158,221,2.12,45,0.43,20,5,2,2,2
2,Italy,83,20,46,17,114,128,1.54,36,0.43,18,4,2,1,1
3,Argentina,77,22,46,9,84,131,1.70,52,0.68,16,2,3,0,0
4,England,62,19,26,17,44,79,1.27,36,0.58,14,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,China PR,3,3,0,0,0,0,0.00,9,3.00,1,0,0,0,0
77,Canada,3,3,0,0,0,0,0.00,5,1.67,1,0,0,0,0
78,Bosnia and Herzegovina,3,2,1,0,1,4,1.33,3,1.00,1,0,0,0,0
79,Zaire,3,3,0,0,0,0,0.00,14,4.67,1,0,0,0,0


# Host countries

In [90]:
host = matches[['Year', 'Stadium', 'City', 'MatchID']]

In [91]:
main_host = winners[['Year', 'Country']]

In [92]:
host = host.merge(main_host, left_on='Year', right_on='Year', how='left')

In [93]:
host = host.groupby('Country')['MatchID'].nunique().reset_index()

In [94]:
host.columns = ['team', 'matches_host']

In [95]:
matches_info = matches_info.merge(host, left_on='team', right_on='team', how='left')

In [96]:
matches_info = matches_info.fillna(0)

In [97]:
matches_info['matches_host'] = matches_info['matches_host'].astype(int)

In [98]:
matches_info.to_csv('matches_info.csv', index=False)